# This file currently exist to test the BGG API. The code is not yet in any particular order

Eventual information I hope to obtain:
 * Take about 100 comments from each to find what people do or don't like about certain games
   * See if that has any relation to the game's categories or mechanism
   * (Not sure if possible) See how these people have rated other games, find similar tastes
 * Take the score of a game and see how it relates to player count, age, and mechanism
   * Are some mechanisms consistently more popular? Are some more divisive?
   * Are newer games rated more highly? Have games gotten better?

In [2]:
import requests
from lxml import etree
from bs4 import BeautifulSoup

In [3]:
# Testing with the information for Catan
params = {'id' : '13', 'ratingcomments' : '1', 'comments' : '1', 'pagesize' : '100', 'page':[3,4], 'stats' :'1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
soup = BeautifulSoup(response.text, "xml")
soup

<?xml version="1.0" encoding="utf-8"?>
<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item id="13" type="boardgame">
<thumbnail>https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ7H8yw__thumb/img/8a9HeqFydO7Uun_le9bXWPnidcA=/fit-in/200x150/filters:strip_icc()/pic2419375.jpg</thumbnail>
<image>https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ7H8yw__original/img/xV7oisd3RQ8R-k18cdWAYthHXsA=/0x0/filters:format(jpeg)/pic2419375.jpg</image>
<name sortindex="1" type="primary" value="Catan"/>
<name sortindex="1" type="alternate" value="CATAN"/>
<name sortindex="1" type="alternate" value="Catan (Колонизаторы)"/>
<name sortindex="1" type="alternate" value="Catan telepesei"/>
<name sortindex="1" type="alternate" value="Catan: Das Spiel"/>
<name sortindex="1" type="alternate" value="Catan: Die Bordspel"/>
<name sortindex="1" type="alternate" value="Catan: El Juego"/>
<name sortindex="1" type="alternate" value="Catan: Gra planszowa"/>
<name sortindex="1" type="alternate" value="Catan: Il

In [4]:
comment_li = []

# Obtaining ten pages of user comments/ratings
for i in range(1,11):
    params = {'id' : '13', 'ratingcomments' : '1', 'comments' : '1', 'pagesize' : '100', 'page':[3,4], 'stats' :str(i)}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    comments = soup.find_all("comment")
    for com in comments:
        comment_li.append(com)

comment_li

[<comment rating="6" username="AgentMeat" value="Whenever I look at my modern board game collection and then at Settlers of Catan, I think this game is done, it's just a relic from the past with some fond memories attached to it, but then I play it and enjoy it all over again. There is something Zen about its whole structure, flow, philosophy. If you don't care about competitive play and you want instead to build and shape the landscape then this is still a strong entry to this date."/>,
 <comment rating="6" username="AgentOddball" value="Another classic that I'm almost always happy to play.  I enjoy attempting to manage the sporadic nature of the resource gain.  You must be ready to change your plan if the situation isn't what you wanted it to be.  Update: Bumping down to 7.  I stand by what I said before, but this one is starting to show its gateway status for me.  There are deeper titles on my shelf more worthy of play.  Update: Bumping down to 6.  It's a fine gateway game, but I do

In [5]:
# Obtain game categories and mechanisms
cat = soup.find_all('link')

for x in cat:
    if x['type'] == 'boardgamecategory':
        print(x['value'])
    elif x['type'] == 'boardgamemechanic':
        print(x['value'])

Economic
Negotiation
Dice Rolling
Hexagon Grid
Income
Modular Board
Network and Route Building
Race
Random Production
Trading
Variable Set-up


In [213]:
# Find the votes with respect to the number of players
results = soup.find_all('results', {'numplayers':'1'})

# general text for vote counts
soup2 = BeautifulSoup(str(results[0]), 'xml')
print(soup2)

# Specific value of the recommendation for 1 player
# Side note, this is a troll vote as Catan requires at least 3 player
soup2.find('result', {'value' :'Best'})['numvotes']

<?xml version="1.0" encoding="utf-8"?>
<results numplayers="1">
<result numvotes="1" value="Best"/>
<result numvotes="3" value="Recommended"/>
<result numvotes="1101" value="Not Recommended"/>
</results>


'1'

In [190]:
# I can also obtain the stats for a game
params = {'id' : '13', 'ratingcomments' : '1', 'comments' : '1', 'pagesize' : '100', 'page' :'1', 'stats' :'1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)

<?xml version="1.0" encoding="utf-8"?>
<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item id="13" type="boardgame">
<thumbnail>https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ7H8yw__thumb/img/8a9HeqFydO7Uun_le9bXWPnidcA=/fit-in/200x150/filters:strip_icc()/pic2419375.jpg</thumbnail>
<image>https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ7H8yw__original/img/xV7oisd3RQ8R-k18cdWAYthHXsA=/0x0/filters:format(jpeg)/pic2419375.jpg</image>
<name sortindex="1" type="primary" value="Catan"/>
<name sortindex="1" type="alternate" value="CATAN"/>
<name sortindex="1" type="alternate" value="Catan (Колонизаторы)"/>
<name sortindex="1" type="alternate" value="Catan telepesei"/>
<name sortindex="1" type="alternate" value="Catan: Das Spiel"/>
<name sortindex="1" type="alternate" value="Catan: Die Bordspel"/>
<name sortindex="1" type="alternate" value="Catan: El Juego"/>
<name sortindex="1" type="alternate" value="Catan: Gra planszowa"/>
<name sortindex="1" type="alternate" value="Catan: Il

In [6]:
# We look at certain users from the comments nad see what other games they might like

params = {'name' : 'AgentMeat', 'hot' : '1', 'top' : '1', 'domain' : 'boardgame', 'page' :'1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/user?', params)
s = BeautifulSoup(response.text, "xml")
s

<?xml version="1.0" encoding="utf-8"?>
<user id="2457479" name="AgentMeat" termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">
<firstname value=""/> <lastname value=""/> <avatarlink value="https://cf.geekdo-static.com/avatars/avatar_id157882.jpg"/> <yearregistered value="2020"/> <lastlogin value="2021-08-23"/> <stateorprovince value="Rheinland-Pfalz"/> <country value="Germany"/> <webaddress value=""/> <xboxaccount value=""/> <wiiaccount value=""/> <psnaccount value=""/> <battlenetaccount value=""/> <steamaccount value=""/> <traderating value="0"/> <marketrating value="0"/>
</user>